In [10]:
import tiktoken
import torch
import torch.nn as nn

import pandas as pd
import sys

sys.path.append('/Users/askarembulatov/Github_projects/NLP_task/NLP/NLP_Learning')

from module import *
from functions import *
from dataloader import *
from preprocessing import *


torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)

ModuleNotFoundError: No module named 'tiktoken'

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "num_head": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "dropout": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

In [ ]:
text_data = read_csv_text()

In [ ]:
train_loader, val_loader = create_train_test_sample(text_data, GPT_CONFIG_124M)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(123)

model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", num_epochs=num_epochs
)

Epoch 0 Step 0 Train loss 9.812215699089897 Val loss 9.924274444580078
Epoch 0 Step 5 Train loss 8.092656400468615 Val loss 8.30355453491211
Every effort moves you, the, the,
Epoch 1 Step 10 Train loss 6.7108720143636065 Val loss 7.074471473693848
Epoch 1 Step 15 Train loss 6.06429926554362 Val loss 6.566693305969238
Every effort moves you, and, and,
Epoch 2 Step 20 Train loss 5.811701244778103 Val loss 6.538724899291992
Epoch 2 Step 25 Train loss 4.9643246862623425 Val loss 6.310605525970459
Every effort moves youSUPSUPSUPSUP 
Epoch 3 Step 30 Train loss 4.804154449039036 Val loss 6.32169246673584
Epoch 3 Step 35 Train loss 4.183735662036472 Val loss 6.226553440093994
Every effort moves you of the, and in
Epoch 4 Step 40 Train loss 3.6980810695224338 Val loss 6.162346839904785
Every effort moves you know that, and in
Epoch 5 Step 45 Train loss 3.227117273542616 Val loss 6.206955432891846
Epoch 5 Step 50 Train loss 2.756786929236518 Val loss 6.208232879638672
Every effort moves you know

In [ ]:
model.to("cpu")
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
            model=model,
            idx=text_to_token_ids("Every effort moves you", tokenizer),
            max_new_tokens=25,
            context_size=GPT_CONFIG_124M["context_length"]
            )
print("Output text:\n", token_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you?"

"Yes--quite insensible to the irony. She wanted him vindicated--and by me!"


